In [ ]:
import itertools
import json, ast
from sklearn.base import clone
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score
import xgboost as xgb
import numpy as np
import pandas as pd
import joblib
import os, math
import gc
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
train_csv = pd.read_csv('D:/UET/ML/mouse_behavior/data/train.csv') 
test_csv = pd.read_csv('D:/UET/ML/mouse_behavior/data/test.csv')
train_annotation_path = 'D:/UET/ML/mouse_behavior/data/train_annotation'
train_tracking_path = 'D:/UET/ML/mouse_behavior/data/train_tracking'

drop_body_parts =  [
    'headpiece_bottombackleft', 'headpiece_bottombackright', 'headpiece_bottomfrontleft', 'headpiece_bottomfrontright', 
    'headpiece_topbackleft', 'headpiece_topbackright', 'headpiece_topfrontleft', 'headpiece_topfrontright', 
    'spine_1', 'spine_2', 'tail_middle_1', 'tail_middle_2', 'tail_midpoint'
]

In [ ]:
mask_lab = train_csv['lab_id'].str.startswith('MABe22')
mask_behavior = train_csv['behaviors_labeled'].isna() | (train_csv['behaviors_labeled'].str.strip() == "")
mask_drop = mask_lab | mask_behavior

train = train_csv[~mask_drop]
body_parts_list = list(np.unique(train.body_parts_tracked))

In [ ]:
def read_tracking(traintest, lab_id, video_id, pix_per_cm, drop_body_parts=[]):
    """
    Output: DataFrame pvid
    Shape: (num_frames, (num_mice * num_bodyparts *2))
    Mỗi hàm = 1 frame, mỗi cột = toạ độ x/y của 1 bodypart của 1 chuột
    """
    video_path = f"D:/UET/ML/mouse_behavior/data/{traintest}_tracking/{lab_id}/{video_id}.parquet"
    df = pd.read_parquet(video_path)

    if len(np.unique(df.bodypart)) > 5:
        df = df[~df.bodypart.isin(drop_body_parts)]

    # Pivot: frame_idx = idx, columns = mouse_id và (x, y) của bodyparts
    pvid = df.pivot(index="video_frame", columns=["mouse_id", "bodypart"], values=["x", "y"])
    pvid = pvid.reorder_levels([1, 2, 0], axis=1).T.sort_index().T
    pvid.columns = pvid.columns.set_names(["mouse_id", "bodypart", "coord"])

    # Chuấn hoá pixel -> cm
    pvid /= pix_per_cm

    mouse_ids = pvid.columns.get_level_values("mouse_id").unique()
    pvid.attrs["mouse_ids"] = mouse_ids
    pvid.attrs["num_mice"] = len(mouse_ids)
    return pvid

In [ ]:
def create_single_mouse_dfs(pvid, annot=None):
    """
    Trả về: list các DataFrame, mỗi DF ứng với 1 chuột
    """
    single_dfs = []
    mouse_ids = pvid.attrs["mouse_ids"]

    # Annotation self-oriented
    if annot is not None:
        annot_self = annot[annot.agent_id == annot.target_id]
    else:
        annot_self = None

    for mouse_id in mouse_ids:
        # Meta info
        meta = pd.DataFrame({
            "video_frame": pvid.index,
            "agent_id": mouse_id,
            "target_id": mouse_id
        }, index=pvid.index)
        
        # Features tracking của chuột
        mouse_features = pvid[mouse_id].copy()

        # Build labels
        if annot_self is not None:
            subset = annot_self[annot_self.agent_id == mouse_id]

            actions = subset["action"].unique()
            labels = pd.DataFrame(0, index=pvid.index, columns=actions)

            for _, row in subset.iterrows():
                labels.loc[row.start_frame:row.stop_frame, row.action] = 1
        else:
            labels = None

        # Gom vào 1 dataframe
        if labels is not None:
            df_mouse = pd.concat([mouse_features, meta, labels], axis=1)
        else:
            df_mouse = pd.concat([mouse_features, meta], axis=1)

        # Thêm vào list
        single_dfs.append(df_mouse)

    return single_dfs

def create_mouse_pair_dfs(pvid, annot=None):
    """
    Trả về: list các DataFrame, mỗi DF ứng với 1 cặp agent-target
    """
    pair_dfs = []

    # Annotation social-oriented
    if annot is not None:
        annot_pair = annot[annot.agent_id != annot.target_id]
    else:
        annot_pair = None

    # Get unique mouse IDs
    mouse_ids = np.unique(pvid.columns.get_level_values("mouse_id"))

    for agent, target in itertools.permutations(mouse_ids, 2):
        # Meta info
        meta = pd.DataFrame({
            "video_frame": pvid.index,
            "agent_id": agent,
            "target_id": target
        }, index=pvid.index)

        # Features tracking của agent-target
        pair_features = pvid.loc[:, pvid.columns.get_level_values("mouse_id").isin([agent, target])]

        mapping = {agent: "A", target: "B"}
        pair_features.columns = pd.MultiIndex.from_tuples(
            [(mapping[m], bodypart, coord) for m, bodypart, coord in pair_features.columns]
        )

        # Build labels
        if annot_pair is not None:
            subset = annot_pair[(annot_pair.agent_id==agent) & (annot_pair.target_id==target)]
            actions = subset["action"].unique()
            labels = pd.DataFrame(0, index=pvid.index, columns=actions)
            for _, row in subset.iterrows():
                labels.loc[row.start_frame:row.stop_frame, row.action] = 1
            df_mouses = pd.concat([pair_features, meta, labels], axis=1)
        else:
            df_mouses = pd.concat([pair_features, meta], axis=1)

        pair_dfs.append(df_mouses)

    return pair_dfs

In [ ]:
def calculate_centers(df):
    """
    Ensure that 'body_center' exists for all mice or bodyparts combinations.
    Handles both 2-level (bodypart, coord) and 3-level (mouse_id, bodypart, coord) columns.
    
    Fallback logic:
    1. If nose and tail_base exist → midpoint(nose, tail_base)
    2. Else if head and tail_base exist → midpoint(head, tail_base)
    3. Else if only tail_base exists → use tail_base
    4. Else → cannot compute body_center
    """
    cols = df.columns

    # --- 2-level columns (bodypart, coord) ---
    if cols.nlevels == 2:
        if ("body_center", "x") not in df.columns or ("body_center", "y") not in df.columns:
            if ("nose", "x") in df.columns and ("tail_base", "x") in df.columns:
                df[("body_center", "x")] = (df[("nose", "x")] + df[("tail_base", "x")]) / 2
                df[("body_center", "y")] = (df[("nose", "y")] + df[("tail_base", "y")]) / 2
            elif ("head", "x") in df.columns and ("tail_base", "x") in df.columns:
                df[("body_center", "x")] = (df[("head", "x")] + df[("tail_base", "x")]) / 2
                df[("body_center", "y")] = (df[("head", "y")] + df[("tail_base", "y")]) / 2
            elif ("tail_base", "x") in df.columns:
                df[("body_center", "x")] = df[("tail_base", "x")]
                df[("body_center", "y")] = df[("tail_base", "y")]
            else:
                # no valid bodyparts → fill NaN
                df[("body_center", "x")] = np.nan
                df[("body_center", "y")] = np.nan

    # --- 3-level columns (mouse_id, bodypart, coord) ---
    elif cols.nlevels == 3:
        mice = sorted(list(set(c[0] for c in cols)))

        for m in mice:
            has_body_center = ((m, "body_center", "x") in cols) and ((m, "body_center", "y") in cols)
            if not has_body_center:
                if ((m, "nose", "x") in cols) and ((m, "tail_base", "x") in cols):
                    df[(m, "body_center", "x")] = (df[(m, "nose", "x")] + df[(m, "tail_base", "x")]) / 2
                    df[(m, "body_center", "y")] = (df[(m, "nose", "y")] + df[(m, "tail_base", "y")]) / 2
                elif ((m, "head", "x") in cols) and ((m, "tail_base", "x") in cols):
                    df[(m, "body_center", "x")] = (df[(m, "head", "x")] + df[(m, "tail_base", "x")]) / 2
                    df[(m, "body_center", "y")] = (df[(m, "head", "y")] + df[(m, "tail_base", "y")]) / 2
                elif ((m, "tail_base", "x") in cols):
                    df[(m, "body_center", "x")] = df[(m, "tail_base", "x")]
                    df[(m, "body_center", "y")] = df[(m, "tail_base", "y")]
                else:
                    df[(m, "body_center", "x")] = np.nan
                    df[(m, "body_center", "y")] = np.nan
    return df

def calculate_speed_lag(df, part, fps, lag=10, mouse=None):
    cols = df.columns
    if mouse is not None:
        x = df[(mouse, part, "x")]
        y = df[(mouse, part, "y")]
    else:
        x = df[(part, "x")]
        y = df[(part, "y")]

    if x.isna().all() or y.isna().all():
        # all missing → return zeros
        return pd.Series(0, index=df.index)

    dx = x.diff(lag)
    dy = y.diff(lag)
    speed = np.sqrt(dx**2 + dy**2) * fps
    return speed.fillna(0)

# Tính các thống kê của 1 đại lượng theo nhiều cửa sổ thời gian
def calculate_window_stats(df, metric, name, fps, scales=[20, 40, 60, 80]):
    """
    Thêm rolling statistics cho bất kỳ series nào.
    
    metric : pd.Series (ví dụ speed, distance, curvature...)
    fps    : frames_per_second
    scales : list window sizes quy đổi theo 30fps → mặc định [20,40,60,80]
    """
    res = pd.DataFrame(index=df.index)
    for scale in scales:
        ws = max(1, int(round(scale * float(fps) / 30)))
        roll = metric.rolling(ws, min_periods=max(1, ws//4))

        res[f"{name}_mean_{scale}"] = roll.mean()
        res[f"{name}_std_{scale}"]  = roll.std()
        res[f"{name}_min_{scale}"]  = roll.min()
        res[f"{name}_max_{scale}"]  = roll.max()

    return res

In [ ]:
def build_single_features(single_mouse_df, meta_fps):
    df_features = pd.DataFrame(index=single_mouse_df.index)
    
    # Get actual bodypart columns
    bodyparts = [col[0] for col in single_mouse_df.columns if isinstance(col, tuple)]
    
    single_mouse_df = calculate_centers(single_mouse_df)
    
    # === Shape and Position Features ===
    # Euclidean distances between bodyparts
    distances = pd.DataFrame({
        f"{p1}+{p2}": np.sqrt(
            (single_mouse_df[(p1, "x")] - single_mouse_df[(p2, "x")])**2 + 
            (single_mouse_df[(p1, "y")] - single_mouse_df[(p2, "y")])**2
        )
        for p1, p2 in itertools.combinations(bodyparts, 2)
    }, index=single_mouse_df.index)
    
    df_features = pd.concat([df_features, distances], axis=1)
    
    # Elongation (only if required bodyparts exist)
    if "nose" in bodyparts and "tail_base" in bodyparts and "ear_left" in bodyparts and "ear_right" in bodyparts:
        df_features["elong"] = distances["nose+tail_base"] / distances["ear_left+ear_right"]
    
    # Body angle (only if nose, tail_base, body_center exist)
    if all(bp in bodyparts for bp in ["nose", "tail_base", "body_center"]):
        v1_x = single_mouse_df[("nose","x")] - single_mouse_df[("body_center","x")]
        v1_y = single_mouse_df[("nose","y")] - single_mouse_df[("body_center","y")]
        v2_x = single_mouse_df[("tail_base","x")] - single_mouse_df[("body_center","x")]
        v2_y = single_mouse_df[("tail_base","y")] - single_mouse_df[("body_center","y")]
        df_features["body_angle"] = (v1_x*v2_x + v1_y*v2_y) / (np.sqrt(v1_x**2+v1_y**2) * np.sqrt(v2_x**2+v2_y**2) + 1e-6)
    
    # === Movement Features ===
    if "body_center" in bodyparts:
        df_features["speed"] = np.sqrt(
            single_mouse_df[("body_center", "x")].diff()**2 +
            single_mouse_df[("body_center", "y")].diff()**2
        )
        df_features["accelerate"] = df_features["speed"].diff()
    
    # Speed lag features for available bodyparts
    for p in ["body_center", "ear_left", "ear_right"]:
        if p in bodyparts:
            df_features[f"speed_{p}_lag_10"] = calculate_speed_lag(single_mouse_df, p, fps=meta_fps)
    
    # Rolling stats for available bodyparts
    for part in bodyparts:
        speed = np.sqrt(single_mouse_df[(part, "x")].diff()**2 + single_mouse_df[(part, "y")].diff()**2) * float(meta_fps)
        res = calculate_window_stats(single_mouse_df, speed, f"speed_{part}", fps=meta_fps)
        df_features = pd.concat([df_features, res], axis=1)
    
    # Curvature for available bodyparts
    for p in ["body_center", "ear_left", "ear_right"]:
        if p in bodyparts:
            lag = max(1, int(round(10 * float(meta_fps) / 30)))
            shifted_x = single_mouse_df[(p,"x")].shift(lag)
            shifted_y = single_mouse_df[(p,"y")].shift(lag)
            df_features[f"curvature_{p}_lag_{lag}"] = np.sqrt((shifted_x - single_mouse_df[(p,"x")])**2 + 
                                                              (shifted_y - single_mouse_df[(p,"y")])**2)

    df_features = df_features.T.drop_duplicates().T
    
    return df_features.fillna(0)


def build_pair_features(mouse_pair_df, meta_fps):
    df_features = pd.DataFrame(index=mouse_pair_df.index)
    
    # Get bodyparts for both mice
    bodyparts_A = [c[1] for c in mouse_pair_df.columns if isinstance(c, tuple) and c[0] == "A"]
    bodyparts_B = [c[1] for c in mouse_pair_df.columns if isinstance(c, tuple) and c[0] == "B"]
    
    mouse_pair_df = calculate_centers(mouse_pair_df)
    
    # Pairwise distances
    distances = pd.DataFrame({
        f"A_{p1}+B_{p2}": np.sqrt(
            (mouse_pair_df[("A", p1, "x")] - mouse_pair_df[("B", p2, "x")])**2 +
            (mouse_pair_df[("A", p1, "y")] - mouse_pair_df[("B", p2, "y")])**2
        )
        for p1, p2 in itertools.product(bodyparts_A, bodyparts_B)
    }, index=mouse_pair_df.index)
    
    df_features = pd.concat([df_features, distances], axis=1)
    
    # Relative orientation (only if nose and tail_base exist for both)
    if all(bp in bodyparts_A for bp in ["nose","tail_base"]) and all(bp in bodyparts_B for bp in ["nose","tail_base"]):
        vec_A_x = mouse_pair_df[("A", "nose", "x")] - mouse_pair_df[("A", "tail_base", "x")]
        vec_A_y = mouse_pair_df[("A", "nose", "y")] - mouse_pair_df[("A", "tail_base", "y")]
        vec_B_x = mouse_pair_df[("B", "nose", "x")] - mouse_pair_df[("B", "tail_base", "x")]
        vec_B_y = mouse_pair_df[("B", "nose", "y")] - mouse_pair_df[("B", "tail_base", "y")]
        df_features["relative_orientation"] = (vec_A_x*vec_B_x + vec_A_y*vec_B_y) / (
            np.sqrt(vec_A_x**2 + vec_A_y**2) * np.sqrt(vec_B_x**2 + vec_B_y**2) + 1e-6
        )
    else: 
        vec_A_x = mouse_pair_df[("A", "head", "x")] - mouse_pair_df[("A", "tail_base", "x")]
        vec_A_y = mouse_pair_df[("A", "head", "y")] - mouse_pair_df[("A", "tail_base", "y")]
        vec_B_x = mouse_pair_df[("B", "head", "x")] - mouse_pair_df[("B", "tail_base", "x")]
        vec_B_y = mouse_pair_df[("B", "head", "y")] - mouse_pair_df[("B", "tail_base", "y")]
        df_features["relative_orientation"] = (vec_A_x*vec_B_x + vec_A_y*vec_B_y) / (
            np.sqrt(vec_A_x**2 + vec_A_y**2) * np.sqrt(vec_B_x**2 + vec_B_y**2) + 1e-6
        )
    
    # Center distance and approach
    if "body_center" in bodyparts_A and "body_center" in bodyparts_B:
        dist_center = np.sqrt(
            (mouse_pair_df[("A", "body_center", "x")] - mouse_pair_df[("B", "body_center", "x")])**2 +
            (mouse_pair_df[("A", "body_center", "y")] - mouse_pair_df[("B", "body_center", "y")])**2
        )
        approach = dist_center.diff().fillna(0)
        df_features["approach_A"] = approach
        df_features["approach_B"] = approach
        
        # Relative distance stats
        res = calculate_window_stats(mouse_pair_df, dist_center**2, "center_distance", fps=meta_fps)
        df_features = pd.concat([df_features, res], axis=1)
        
        # Relative speed
        speed_A = calculate_speed_lag(mouse_pair_df, "body_center", meta_fps, mouse="A")
        speed_B = calculate_speed_lag(mouse_pair_df, "body_center", meta_fps, mouse="B")
        df_features["speed_A_lag_10"] = speed_A
        df_features["speed_B_lag_10"] = speed_B
        df_features["relative_speed_A_B_lag_10"] = (speed_A - speed_B).abs()

    df_features = df_features.T.drop_duplicates().T
    
    return df_features.fillna(0)

In [ ]:
def generate_mouse_data(dataset, traintest, drop_body_parts=[]):
    """
    Tạo DataFrames cho mỗi chuột và mỗi cặp chuột trong 1 video
    Trả về danh sách DataFrames
    """
    all_single = []
    all_pair = []

    for _, row in dataset.iterrows():
        # Meta info
        lab_id = row.lab_id
        video_id = row.video_id
        pix_per_cm = row.pix_per_cm_approx
        meta_fps = row.frames_per_second

        if lab_id.startswith('MABe22') or type(row.behaviors_labeled) != str: 
            continue

        # Đọc tracking
        pvid = read_tracking(traintest, lab_id, video_id, pix_per_cm, drop_body_parts=drop_body_parts)
        
        # Đọc annotation
        annot = None
        if traintest == "train":
            annot_path = f"/kaggle/input/MABe-mouse-behavior-detection/{traintest}_annotation/{lab_id}/{video_id}.parquet"

            if not os.path.exists(annot_path):
                print(f"Skipping video {video_id} (annotation not found)")
                continue

            annot = pd.read_parquet(annot_path)
            
        # Tạo DataFrame cho từng chuột
        sdfs = create_single_mouse_dfs(pvid, annot)

        for df_single in sdfs:
            df_features = build_single_features(df_single, meta_fps)
            df_meta = df_single[["video_frame","agent_id","target_id"]]

            if annot is not None: 
                df_labels = df_single[[c for c in df_single.columns 
                        if (not isinstance(c, tuple)) and (c not in df_meta.columns)]]
                
            else:
                df_labels = pd.DataFrame(index=df_single.index)

            all_single.append([df_meta, df_features, df_labels])

        # Nếu có nhiều hơn 1 chuột, tạo DataFrame cho từng cặp chuột
        if pvid.attrs["num_mice"] > 1:
            pdfs = create_mouse_pair_dfs(pvid, annot)
            for df_pair in pdfs:
                df_features = build_pair_features(df_pair, meta_fps)
                df_meta = df_pair[["video_frame","agent_id","target_id"]]
                
                if annot is not None:
                    df_labels = df_pair[[c for c in df_pair.columns 
                           if (not isinstance(c, tuple)) and (c not in df_meta.columns)]]
                else:
                    df_labels = pd.DataFrame(index=df_pair.index)

                all_pair.append([df_meta, df_features, df_labels])

    return all_single.astype(np.float32), all_pair.astype(np.float32)

In [ ]:
def train_binary_classifiers(data_list, mode, model, section, n_splits=5):
    """
    data_list: list of (df_meta, df_feat, df_lab) for each video (one section)
    mode: "single" or "pair"
    model: scikit-learn estimator (cloneable)
    section: int used for saving directory
    n_splits: GroupKFold splits (minimum groups required)

    Returns:
        f1_list: list of (section, action, best_f1)
        thresholds: dict action -> best_threshold
    """

    save_dir = f"models/{section}/{mode}"
    os.makedirs(save_dir, exist_ok=True)

    thresholds = {}
    f1_list = []

    # Gather all actions in this section
    all_actions = set()
    for _, _, df_lab in data_list:
        all_actions.update(df_lab.columns)
    all_actions = sorted(all_actions)

    if len(all_actions) == 0:
        print(f"[Section {section} | {mode}] No actions found, skipping.")
        return f1_list, thresholds

    # ---------------------------------------------------------
    #   TRAIN SEPARATE MODEL FOR EACH ACTION
    # ---------------------------------------------------------
    for action in all_actions:
        print(f"\n[Section {section} | {mode}] Preparing data for action: {action}")

        feat_dfs = []      
        label_arrays = []  
        group_labels = []  
        video_idx = 0

        # -------- Collect per-video data --------
        for vid_idx, (df_meta, df_feat, df_lab) in enumerate(data_list):

            if action not in df_lab.columns:
                continue

            y = df_lab[action].values.astype(np.int8)
            if y.sum() == 0:
                continue

            groups = np.full(len(y), video_idx, dtype=np.int32)

            # Ensure df_feat is DataFrame
            if not isinstance(df_feat, pd.DataFrame):
                df_feat = pd.DataFrame(df_feat)

            feat_dfs.append(df_feat)
            label_arrays.append(y)
            group_labels.append(groups)

            video_idx += 1

        n_videos_for_action = len(feat_dfs)
        if n_videos_for_action == 0:
            print(f"[Section {section} | {mode}] Action '{action}' has no positive videos — skipping.")
            continue

        if n_videos_for_action < n_splits:
            print(f"[Section {section} | {mode}] Action '{action}' only appears in {n_videos_for_action} videos (< {n_splits}), skipping.")
            continue

        # ---------------------------------------------------------
        #   ALIGN COLUMNS (union of all columns) – duplicates removed already upstream
        # ---------------------------------------------------------
        all_cols = sorted({c for df in feat_dfs for c in df.columns})

        for i in range(len(feat_dfs)):
            feat_dfs[i] = feat_dfs[i].reindex(columns=all_cols).fillna(0).astype(np.float32)

        # -------- Stack all videos --------
        X_all_df = pd.concat(feat_dfs, axis=0, ignore_index=True)
        y_all = np.concatenate(label_arrays).astype(np.int8)
        groups_all = np.concatenate(group_labels).astype(np.int32)

        X_all = X_all_df.values 
        del X_all_df, feat_dfs, label_arrays, group_labels
        gc.collect()

        N = len(y_all)
        if X_all.shape[0] != N:
            raise RuntimeError("X_all and y_all size mismatch.")

        # ---------------------------------------------------------
        #   OUT-OF-FOLD PREDICTIONS (GroupKFold)
        # ---------------------------------------------------------
        oof_preds = np.zeros(N, dtype=np.float32)

        unique_groups = np.unique(groups_all)
        gkf_splits = min(n_splits, len(unique_groups))
        gkf = GroupKFold(n_splits=gkf_splits)

        idx = np.arange(N)
        for train_idx, val_idx in gkf.split(idx, y_all, groups_all):
            clf = clone(model)
            clf.fit(X_all[train_idx], y_all[train_idx])
            oof_preds[val_idx] = clf.predict_proba(X_all[val_idx])[:, 1]
            del clf
            gc.collect()

        # ---------------------------------------------------------
        #   THRESHOLD TUNING
        # ---------------------------------------------------------
        best_f1, best_thresh = 0.0, 0.5
        for t in np.linspace(0.1, 0.9, 17):
            preds_bin = (oof_preds >= t).astype(int)
            f1 = f1_score(y_all, preds_bin, zero_division=0)
            if f1 > best_f1:
                best_f1, best_thresh = f1, float(t)

        thresholds[action] = best_thresh
        f1_list.append((section, action, best_f1))

        # ---------------------------------------------------------
        #   FINAL MODEL TRAINING ON ALL DATA
        # ---------------------------------------------------------
        final_clf = clone(model)
        final_clf.fit(X_all, y_all)

        joblib.dump(final_clf, f"{save_dir}/{action}_model.pkl")
        joblib.dump(best_thresh, f"{save_dir}/{action}_threshold.pkl")

        print(f"[Section {section} | {mode}] Action '{action}': {n_videos_for_action} videos, {N} samples, F1={best_f1:.4f}, thr={best_thresh:.3f}")

        del final_clf, X_all, y_all, groups_all, oof_preds
        gc.collect()

    return f1_list, thresholds

In [ ]:
thresholds = {"single": {}, "pair": {}}
f1_list = []
submission_list = []

# Train model
for section in range(1, len(body_parts_list)):
    # Lấy body_parts_tracked trong số 9 bộ của toàn dataset
    body_parts_tracked = body_parts_list[section]

    # Lấy các rows/videos được thu với body_parts_tracked tương ứng
    train_subset = train[train.body_parts_tracked == body_parts_tracked]

    if train_subset.empty:
        print("\nNo videos in this section, skipping...")
        continue

    # Data từng chuột + Data cặp chuột của toàn bộ videos trong subset
    single_data, pair_data = generate_mouse_data(train_subset, "train", drop_body_parts)

    # Tạo model
    trainer = xgb.XGBClassifier(n_estimators=150, learning_rate=0.08, max_depth=4,
                            subsample=0.7, colsample_bytree=0.7, random_state=42)

    # Train XGBoost
    if len(single_data) > 0:
        single_f1_list, single_thresholds = train_binary_classifiers(single_data, "single", trainer, section, n_splits=3)

    if len(pair_data) > 0:
        pair_f1_list, pair_thresholds = train_binary_classifiers(pair_data, "pair", trainer, section, n_splits=3)

    f1_list.extend(single_f1_list)
    f1_list.extend(pair_f1_list)
    thresholds["single"][section] = single_thresholds
    thresholds["pair"][section] = pair_thresholds

In [ ]:
def detect_action_segments(frames, preds, min_length=1):
    """
    frames: np.array các frame index (ví dụ: [0,1,2,3,...])
    preds: np.array nhị phân 0/1 dự đoán theo từng frame
    min_length: độ dài tối thiểu của 1 segment (theo số frame)
    
    return: list[(start_frame, stop_frame)]
    """
    segments = []
    in_segment = False
    start_frame = None

    for i in range(len(preds)):
        if preds[i] == 1 and not in_segment:
            # bắt đầu 1 đoạn mới
            in_segment = True
            start_frame = frames[i]

        elif preds[i] == 0 and in_segment:
            # kết thúc đoạn
            end_frame = frames[i - 1]
            if end_frame - start_frame + 1 >= min_length:
                segments.append((start_frame, end_frame))
            in_segment = False

    # Nếu kết thúc file mà vẫn đang trong segment
    if in_segment:
        end_frame = frames[-1]
        if end_frame - start_frame + 1 >= min_length:
            segments.append((start_frame, end_frame))

    return segments

def create_submission(test_results, video_id): 
    """ 
    test_results: predictions của toàn bộ test videos 
    video_id: ID của video test 
    """ 
    rows = [] 
    for (_, df_meta, df_preds) in test_results:
        frames = df_meta["video_frame"].values

        for action in df_preds.columns: 
            pred_col = df_preds[action].values 
            segments = detect_action_segments(frames, pred_col) 
            
            for (start, stop) in segments: # Lấy agent_id/target_id từ df_meta tương ứng với start frame 
                mask = (df_meta["video_frame"] >= start) & (df_meta["video_frame"] <= stop) 
                agent_ids = df_meta.loc[mask, "agent_id"].unique() 
                target_ids = df_meta.loc[mask, "target_id"].unique() # Nếu single thì agent_id == target_id, nếu pair thì giữ 2 id 
            
                for a_id in agent_ids: 
                    for t_id in target_ids: 
                        rows.append([ 
                            video_id, 
                            a_id, 
                            t_id, 
                            action, 
                            int(start), 
                            int(stop) 
                        ]) 
                    
    submission = pd.DataFrame(rows, columns=["video_id", "agent_id", "target_id", "action", "start_frame", "stop_frame"]) 
    submission.insert(0, "row_id", range(len(submission))) 
    
    return submission

In [ ]:
test = test_csv.copy()
test_videos = test["video_id"].unique()

submission_list = []

for vid in test_videos:
    video_rows = test[test["video_id"] == vid]
    body_parts_tracked = video_rows["body_parts_tracked"].iloc[0]

    # Lấy index section từ body_parts_list (giả sử body_parts_list có sẵn)
    try:
        section = body_parts_list.index(body_parts_tracked)
    except ValueError:
        print(f"Video {vid}: body_parts_tracked not found in body_parts_list, skip")
        continue

    # Load model của section tương ứng
    section_dir = f"/kaggle/working/models/{section}"
    single_actions = [f[:-11] for f in os.listdir(f"{section_dir}/single") if f.endswith("_model.pkl")]
    pair_actions   = [f[:-11] for f in os.listdir(f"{section_dir}/pair")   if f.endswith("_model.pkl")]

    # Tạo single + pair data cho video test
    single_data, pair_data = generate_mouse_data(
        video_rows, 
        "test", 
        drop_body_parts=drop_body_parts
    )

    # Test từng single mouse
    for df_meta, df_feat, _ in single_data:
        df_preds = pd.DataFrame(index=df_feat.index)

        for action in single_actions:
            model_path = f"{section_dir}/single/{action}_model.pkl"
            thresh_path = f"{section_dir}/single/{action}_threshold.pkl"
            
            clf = joblib.load(model_path)
            threshold = joblib.load(thresh_path)
            
            proba = clf.predict_proba(df_feat.values)[:,1]
            preds = (proba >= threshold).astype(int)

            df_preds[action] = preds
            
        submission_list.append((vid, df_meta, df_preds))

    # Test từng pair mouse
    for df_meta, df_feat, _ in pair_data:
        df_preds = pd.DataFrame(index=df_feat.index)

        for action in pair_actions:
            model_path = f"{section_dir}/pair/{action}_model.pkl"
            thresh_path = f"{section_dir}/pair/{action}_threshold.pkl"
            
            clf = joblib.load(model_path)
            threshold = joblib.load(thresh_path)
            
            proba = clf.predict_proba(df_feat.values)[:,1]
            preds = (proba >= threshold).astype(int)

            df_preds[action] = preds
            
        submission_list.append((vid, df_meta, df_preds))

# Tạo submission duy nhất
final_submission_rows = []
for vid in test_videos:
    vid_results = [x for x in submission_list if x[0] == vid]
    if len(vid_results) == 0:
        continue

    submission_df = create_submission(vid_results, vid)
    final_submission_rows.append(submission_df)

final_submission = pd.concat(final_submission_rows, axis=0, ignore_index=True)
    
final_submission.to_csv()
final_submission